In [1]:
import os
import sys

In [2]:
#os.chdir('/home/au643300/DataHandling/')
#sys.path.append('/home/au643300/DataHandling/')
os.environ['HDF5_USE_FILE_LOCKING']="FALSE"

In [3]:
from dask_jobqueue import SLURMCluster
from dask.distributed import Client, as_completed,wait,fire_and_forget,progress
import glob
import xarray as xr
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import importlib
import dask
import zarr
import time
from rechunker import rechunk

In [4]:
def custom_optimize(dsk, keys):
    dsk = dask.optimization.inline(dsk, inline_constants=True)
    return dask.array.optimization.optimize(dsk, keys)
sns.set_theme()
dask.config.set({"distributed.comm.timeouts.tcp": "50s"})
dask.config.set({'distributed.comm.retry.count':3})
dask.config.set({'distributed.comm.timeouts.connect':'25s'})
dask.config.set({"distributed.worker.use-file-locking":False})
zarr.blosc.use_threads = False

Loads the Datahandling package(which is everything in the src folder)

In [5]:
%load_ext autoreload


In [6]:
%autoreload 2

In [7]:
import importlib
from DataHandling.data.toxarr import to_xarr

In [8]:
cluster=SLURMCluster(cores=8,
                     memory="31GiB",
                     processes=2,
                     queue='q64',
                     walltime='0-01:00:00',
                     local_directory='/scratch/$SLURM_JOB_ID',
                     interface='ib0',
                     scheduler_options={'interface':'ib0'},
                     extra=['--resources mem=14',"--lifetime", "50m", "--lifetime-stagger", "4m"]
                    )
                    

In [9]:
client=Client(cluster)

In [10]:
cluster.adapt(minimum=2,maximum=8)

In [11]:
client

Client Scheduler: tcp://10.100.0.1:44974 Dashboard: http://10.100.0.1:8787/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


Constants

In [12]:
test="/home/au643300/NOBACKUP/test/"
files=glob.glob(test+'*.nc')
files=sorted(files)
file_names=[os.path.basename(path) for path in files]
file_names=[file[0:-3] for file in file_names]
file_names

['field.0500', 'field.0501', 'field.0502', 'field.0503']

In [13]:
ex=xr.open_mfdataset(files[0:2],parallel=True,combine='nested',concat_dim='time')
ex.attrs['field']='0501'
ex.to_zarr("/home/au643300/NOBACKUP/test.zarr")
# ex=xr.open_zarr("/home/au643300/NOBACKUP/test.zarr")

ContainsGroupError: path '' contains a group

In [20]:
ex

<xarray.Dataset>
Dimensions:  (time: 4, x: 256, y: 201, z: 256)
Coordinates:
  * time     (time) float64 2.024e+03 2.028e+03 2.032e+03 2.036e+03
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
Data variables:
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>

Nu har jeg et dataset at teste på

Laver field om til filnavn

In [25]:
field='field.'+ex.attrs['field']
field

'field.0501'

In [26]:
start=[field==file for file in file_names]
index=int(np.where(start)[0])
index+=1
index

2

In [18]:
files[index:]

['/home/au643300/NOBACKUP/test/field.0502.nc',
 '/home/au643300/NOBACKUP/test/field.0503.nc']

Nu mangler jeg sådan set bare at den gør det direkte fra de u filer der. Det kræver nok at jeg bruger/sætter mig ind i toxr filen, og får den implementeret her, og evt får den lavet til en delayed også. Det vil så kræve at jeg finder ud af at bruge det med ressourser, og at jeg får ændret ressourser oppe i min cluster

In [19]:
# sync=zarr.sync.ProcessSynchronizer('/home/au643300/NOBACKUP/lock.zarr')
# delayed=[]
# with dask.config.set(array_optimize=custom_optimize):
#     for file in files[index:]:
#         ds=xr.open_mfdataset(file,parallel=True,combine='nested',concat_dim='time')
#         ds.attrs['field']=file[-7:-3]
#         time_new=float(ds.time.data)
#         time_ex=float(ex.time.data[-1])
#         if time_new>=time_ex:
#             save=ds.to_zarr("/home/au643300/NOBACKUP/test.zarr", append_dim="time",synchronizer=sync,compute=False)
#             delayed.append(save)
#         else:
#             raise AssertionError(f'{file} timestep {time_ex} is not above the last timestep in the complete dataset, aborting')
#             break
#     dask.compute(delayed)

adding /home/au643300/NOBACKUP/test/field.0502.nc
adding /home/au643300/NOBACKUP/test/field.0503.nc


Så prøver vi for realz

In [37]:
# test="/home/au643300/NOBACKUP/interim/"
# files=glob.glob(test+'*.nc')
# files=sorted(files)
# ex=xr.open_mfdataset(files[0:2],parallel=True,combine='nested',concat_dim='time')
# ex.attrs['field']=files[1][-7:-3]
# ex.to_zarr("/home/au643300/NOBACKUP/data.zarr")


GroupNotFoundError: group not found at path ''

In [12]:
ex=xr.open_zarr("/home/au643300/NOBACKUP/data.zarr")
ex

<xarray.Dataset>
Dimensions:  (time: 2, x: 256, y: 201, z: 256)
Coordinates:
  * time     (time) float64 1.996e+03 2e+03
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
Data variables:
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
Attributes:
    field:    0494

In [13]:
raw="/home/au643300/DataHandling/data/raw/"
files=glob.glob(raw+'*.u')
files=sorted(files)
file_names=[os.path.basename(path) for path in files]
file_names=[file[0:-2] for file in file_names]

In [14]:
field='field.'+ex.attrs['field']
field

'field.0494'

In [15]:
start=[field==file for file in file_names]
index=int(np.where(start)[0])
index+=1
index

2

In [16]:
# with dask.annotate(resources={'mem': 6}):   
#     with dask.config.set(array_optimize=custom_optimize):
#             a=dask.delayed(DataHandling.data.toxarr.to_xarr)(files[1])


In [17]:
# c=a.expand_dims("time")
# c

In [18]:

# with dask.config.set(array_optimize=custom_optimize):
#     save=a.to_zarr("/home/au643300/NOBACKUP/test.zarr", append_dim="time",compute=False)

In [21]:
# save.compute(optimize_graph=False)

Delayed('_finalize_store-3a360566-010c-4008-933d-55d333932009')

In [16]:
save_del=[]
with dask.annotate(resources={'mem': 6}):
    with dask.config.set(array_optimize=custom_optimize):
        for file in files[index:]:
            data=dask.delayed(to_xarr)(file)
    #         ds.attrs['field']=file[-6:-2]
            save=data.to_zarr("/home/au643300/NOBACKUP/data.zarr", append_dim="time",compute=False)
            save_del.append(save)

In [17]:
n=int(len(save_del)/20)
chunks= [save_del[i * n:(i + 1) * n] for i in range((len(save_del) + n - 1) // n )] 

In [18]:
for chunk in chunks:
    dask.compute(chunk,optimize_graph=False)

ValueError: conflicting sizes for dimension 'time': length 3 on 'pr0.2' and length 4 on 'pr0.025'

In [19]:
ds=xr.open_zarr("/home/au643300/NOBACKUP/data.zarr")
ds

<xarray.Dataset>
Dimensions:  (time: 4, x: 256, y: 201, z: 256)
Coordinates:
  * time     (time) float64 1.996e+03 2e+03 2.008e+03 2.028e+03
  * x        (x) float64 0.0 0.04688 0.09375 0.1406 ... 11.81 11.86 11.91 11.95
  * y        (y) float64 2.0 2.0 2.0 1.999 ... 0.00111 0.0004934 0.0001234 0.0
  * z        (z) float64 -3.0 -2.977 -2.953 -2.93 ... 2.906 2.93 2.953 2.977
Data variables:
    pr0.025  (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.2    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr0.71   (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    pr1      (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    u_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    v_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>
    w_vel    (time, x, y, z) float64 dask.array<chunksize=(1, 256, 201, 256), meta=np.ndarray>